In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import spline
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [58]:
same = lambda x : x

def linear(x, derivative=False):
    return np.ones_like(x) if derivative else x

def sigmoid(x, derivative=False):
    if derivative:
        y = sigmoid(x)
        return y*(1 - y)
    return 1.0/(1.0 + np.exp(-x))

def tanh(x, derivative=False):
    if derivative:
        y = tanh(x)
        return 1 - y**2
    return (np.exp(x) - np.exp(-x))/(np.exp(x) + np.exp(-x))

def relu(x, derivative=False):
    if derivative:
        return np.where(x <= 0, 0, 1)
    return np.maximum(0, x)

def step(x):
    return np.where(x>0,1,-1)

In [137]:
class Layer():
    def __init__(self, input_dim, output_dim, activation=linear):
        self.input = None
        self.weights = np.random.randn(output_dim, input_dim)
        self.biases = np.random.randn(1, output_dim)
        self.activation = activation
        self._activ_inp, self._activ_out = None, None
        self.dweights, self.dbiases = None, None

class NeuralNetwork():
    def __init__(self):
        self.layers = []
        
    def fit(self, x, y, learning_rate=1e-3, epochs=100, verbose=10):
        '''
            Função de treinamento
            Params:
                x - entradas
                y - saídas esperadas
                learning_rate - taxa de aprendizagem
                epochs - quantidade de épocas de treinamento
                verbose - passos para verificação da época
        '''
        for epoch in range(epochs+1):
            for x_i, y_i in zip(x, y):
                x_i =  x_i.reshape(1, x.shape[1])
                y_pred = self.__feedforward(x_i)

                erro = y_i - y_pred
                self.layers[-1].weights += learning_rate * np.dot(erro.T, x_i)
                self.layers[-1].biases += learning_rate * erro[0]
            
            if epoch % verbose == 0:
                print("epoch: {0:=4}/{1}".format(epoch, epochs))
    
    def add_layer(self, layer):
        '''
            Método para adição de layers ao modelo
            Params:
                layer - Layer a se adicionada ao modelo
        '''
        self.layers.append(layer)
    
    def __feedforward(self, x):
        '''
            Método de Feedforward
            Params:
                x - dados de entrada
        '''
        self.layers[0].input = x
        for current_layer, next_layer in zip(self.layers, self.layers[1:]+[Layer(0,0)]):
            y = np.dot(current_layer.input, current_layer.weights.T) + current_layer.biases
            current_layer._activ_inp = y
            current_layer._activ_out = next_layer.input = current_layer.activation(y)
        return self.layers[-1]._activ_out
    
    def _forward_prediction(self, x):
        y = []
        
        for layer in self.layers:
            z = np.dot(x, layer.weights.T) + layer.biases
            y = layer.activation(z)
        return y
    
    def prediction(self, x):
        y_pred = []
        
        for data in x:
            y_pred.append(self.___feedforward(data))
            
        return np.array(y_pred)
    
    def evaluate(self,y_pred, y, func=same, dtype=int):
        score = 0
        total = 100.0/y.shape[0]
        # mesmo tipo de saida
        y_pred = y_pred.astype(dtype)
        y = y.astype(dtype)

        for y_i, y_pred_i in zip(y,y_pred):
            y_pred_i = func(y_pred_i)
            if np.array_equal(y_i,y_pred_i[0]):
                score+=1

        return score*total
    
    def __backprop(self, y, y_pred):
        last_delta = self.cost_func(y, y_pred, derivative=True)
        for layer in reversed(self.layers):
            dactivation = layer.activation(layer._activ_inp, derivative=True)*last_delta
            last_delta = np.dot(dactivation, layer.weights)
            layer.dweights = np.dot(dactivation.T, layer.input)
            layer.dbiases = 1.0*dactivation.sum(axis=0, keepdims=True)
        
        for layer in reversed(self.layers):
            layer.weights = layer.weights - self.learning_rate*layer.dweights
            layer.biases = layer.biases - self.learning_rate*layer.dbiases

In [144]:
nn = NeuralNetwork()

x = []
y = []

for line in open('./data_x.txt', 'r').readlines():
    x.append([float(num) for num in line.split(',')])

for line in open('./data_y.txt', 'r').readlines():
    y.append([int(num) for num in line.split(',')])

x = np.array(x[:int(len(x) * 0.8)])
y = np.array(y[:int(len(y) * 0.8)])

x_teste = np.array(x[int(len(x) * 0.8):])
y_teste = np.array(y[int(len(y) * 0.8):])

nn.add_layer(Layer(input_dim=x.shape[1], output_dim=y.shape[1], activation=step))

nn.fit(x, y, learning_rate=0.001 , epochs=200, verbose=100)

y_pred = nn.prediction(x_teste)

print(nn.evaluate(y_pred, y_teste), "%")


epoch:    0/200
epoch:  100/200
epoch:  200/200
100.0 %
